In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
from model.mrcnn.config import Config
from model.mrcnn import utils
import model.mrcnn.model as modellib
from model.mrcnn import visualize
from model.mrcnn.model import log
from model.mrcnn import data_generator as datagen
from model.mrcnn import image_meta as meta
from model.mrcnn import dataset

%matplotlib inline 

In [ ]:
from data.pascalvoc.pascal_frcnn import PascalVOCDataset

In [ ]:
# Directory to save logs and trained model
MODEL_DIR = os.path.join(os.environ['HOME'], 'partial_experiments')

# Local path to trained weights file MEH
COCO_MODEL_PATH = os.path.join(os.environ['HOME'], "partial-labels/experiments/frcnn", "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

In [ ]:
from keras.preprocessing import image as k_image
from keras.applications.imagenet_utils import preprocess_input

In [ ]:
from collections import defaultdict

from pprint import pprint

In [ ]:
class PascalConfig(Config):
    """
    Configuration for training on the Pascal dataset.
    Derives from the base Config class and overrides values specific to Pascal
    """
    # Give the configuration a recognizable name
    NAME = "pascal"

    # Train on 1 GPU and 8 images per GPU. We can put multiple images on each
    # GPU because the images are small. Batch size is 8 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 8

    # Number of classes (including background)
    NB_CLASSES = 20 + 1

    # Use small images for faster training. Set the limits of the small side
    # the large side, and that determines the image shape.
    IMAGE_MIN_DIM = 448
    IMAGE_MAX_DIM = 448

    # Use smaller anchors because our image and objects are small
    RPN_ANCHOR_SCALES = (64, 128, 256)  # anchor side in pixels

    # Reduce training ROIs per image because the images are small and have
    # few objects. Aim to allow ROI sampling to pick 33% positive ROIs.
    TRAIN_ROIS_PER_IMAGE = 32

    # Use a small epoch since the data is simple
    STEPS_PER_EPOCH = 100

    # use small validation steps since the epoch is small
    VALIDATION_STEPS = 5
    
    # old config
    IMG_SIZE = 448
    DATASET_PATH = '/local/DEEPLEARNING/pascalvoc/VOCdevkit/VOC2007'
    NB_CHANNELS = 3
    
config = PascalConfig()
config.display()

In [ ]:
def get_ax(rows=1, cols=1, size=8):
    """
    Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

## Pascal Dataset

In [ ]:
# Training dataset
dataset_train = PascalVOCDataset()
dataset_train.load_pascal(config.DATASET_PATH, 8, 'trainval', config)
dataset_train.prepare()

In [ ]:
# Validation dataset
dataset_val = PascalVOCDataset()
dataset_val.load_pascal(config.DATASET_PATH, 8, 'test', config)
dataset_val.prepare()

In [ ]:
# Load and display random samples
image_ids = np.random.choice(dataset_val.image_ids, 4)
print(image_ids)

In [ ]:
#image_ids = [55102]
for image_id in image_ids:
    print('image id %s' % image_id)
    img = dataset_val.load_image(image_id)
    bboxes, class_ids = dataset_val.load_bboxes(image_id)
    bboxes = np.array(bboxes)
    print(bboxes.shape)
    print(bboxes[0])
    
    class_ids = np.array(class_ids)
    print(class_ids.shape)
    
    visualize.display_instances(img, bboxes, class_ids, dataset_val.class_names, figsize=(8, 8))